In [1]:
# # Run the code below for setting up google colab enviroment
# # install condacolab
# !pip install -q condacolab
# import condacolab
# condacolab.install()

# # copy from github
# !git clone https://github.com/valence-labs/mtl_summer_school_2024.git

# # change working directory
# import os
# os.chdir("mtl_summer_school_2024/Lab1")

# # install dependencies
# !mamba env update -n base -f env.yml

## Public Transcriptomics data
build from https://github.com/cmap/cmapPy/blob/master/tutorials/cmapPy_pandasGEXpress_tutorial.ipynb

## Batch effect handling
concept, then center scale on controls + reading for those interested

## Comparison to control and characterization

Log fold change and GO enrichment... (if time permits)

Pointing out that _this_ 


## Embedding & comparing effect of small molecules and genetic perturbations
e.g. for target deconvolution & detecting off target effects

still have not decided... everything that is good at embedding L1000 is complicated / bringing in a lot more info. 